Notebook to extract fragment of a database for tests.

In [8]:
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import sessionmaker

import sys
sys.path.append('..')

from tracks_interactions.db.db_model import CellDB, TrackDB,Base
from tracks_interactions.db.db_functions import get_descendants

In [9]:
# define pathways

org_db_path = r'D:\kasia\tracking\E6_exp\double_segmentation_ultrack\Exp6_gardener.db'

new_db_path = r'../tests/fixtures/db_2tables_test.db'

In [10]:
engine = create_engine(f'sqlite:///{org_db_path}')
session = sessionmaker(bind=engine)()

new_engine = create_engine(f'sqlite:///{new_db_path}')
Base.metadata.create_all(new_engine)
new_session = sessionmaker(bind=new_engine)()

In [11]:
# get all the descendants of the selected roots
# don't ask for not roots as their parameters will be incorrect in the test database
selected_roots = [37401,20422]
selected_track_id_list = []

for root in selected_roots:
    selected_track_id_list.extend([x.track_id for x in get_descendants(session,root)])

selected_track_id_list

[37401, 37402, 37403, 20422, 20423, 20424, 20425, 20426, 20427, 20428]

In [ ]:
# copy the selected tracks and cells to the new database

for selected_track_id in selected_track_id_list:
    
    cells = session.query(CellDB).filter(CellDB.track_id == selected_track_id).all()
    tracks = session.query(TrackDB).filter(TrackDB.track_id == selected_track_id).all()

    for cell in cells:
        
        # Create a new instance of CellDB
        new_cell = CellDB()
        
        # Deep copy
        for key, value in inspect(cell).attrs.items():
            setattr(new_cell, key, value.value)

        new_session.add(new_cell)

    for track in tracks:

        # Create a new instance of TrackDB
        new_track = TrackDB()

        # Deep copy
        for key, value in inspect(track).attrs.items():
            setattr(new_track, key, value.value)
        
        new_session.add(new_track)

    new_session.commit()

In [7]:
# clean up
session.close()
new_session.close()